In [65]:
import sqlite3 as lite
import math
import pandas as pd
import mysql.connector
import yaml

cfg = yaml.safe_load(open('_inc.yaml'))

cnx = mysql.connector.connect(user=cfg['mysql']['user'], password=cfg['mysql']['pwd'],
                           host=cfg['mysql']['server'], database=cfg['mysql']['db'])


In [66]:
users = pd.read_sql_query('SELECT * FROM tblReview;', cnx)

In [67]:
reviews = users.drop(['gameID', 'movieID', 'tvShowID', 'text', 'date', 'thumbsUp', 'thumbsDown'], axis = 1)


In [68]:
reviews = reviews[3:].reset_index()
print reviews['uniqueID'].max()
reviews.head(5)


27864


,index,author,publication,score,reviewType,uniqueID
0,3,,Areajugones,69,c,1
1,4,,Hobby Consolas,67,c,1
2,5,,GameGrin,60,c,1
3,6,,GameSpew,60,c,1
4,7,,Digitally Downloaded,50,c,1


In [69]:
def pearson_similarity(item1, item2):
    
 
    num_keys = 0
    numerator = 0
    denominator = 0
    num_keys_item1 = 0
    num_keys_item2 = 0
    ratings_total_item1 = 0
    ratings_total_item2 = 0
    rating_diff_item1 = 0
    rating_diff_item2 = 0
    total_diff_item1_squared = 0
    total_diff_item2_squared = 0
    
    for key in item1:
        if key in item2:
            num_keys += 1
    if num_keys < 3:
        #print 'not enough overlap between items'
        return 0
    
    
    for key in item1:
        num_keys_item1 += 1
        ratings_total_item1 += item1[key]
    average_rating_item1 = ratings_total_item1 / num_keys_item1
    
    #print 'Item1 average rating: ', average_rating_item1
        
    for key in item2:
        num_keys_item2 += 1
        ratings_total_item2 += item2[key]
    average_rating_item2 = ratings_total_item2 / num_keys_item2
    
    #print 'Item2 average rating: ', average_rating_item2
    
    for key in item1:
        if key in item2:
            #print key, item1[key], item2[key]
            rating_diff_item1 = item1[key] - average_rating_item1
            rating_diff_item2 = item2[key] - average_rating_item2
            both_diff = rating_diff_item1 * rating_diff_item2
 
            numerator += both_diff
            
            total_diff_item1_squared += rating_diff_item1 ** 2
            total_diff_item2_squared += rating_diff_item2 ** 2
    
    denominator = math.sqrt(total_diff_item1_squared) * math.sqrt(total_diff_item2_squared)
    
    if num_keys == 0:
        return 0
    
    if denominator == 0:
        return 0
    else:
        return numerator/denominator
        

In [70]:
similarity_test = {}
similarity_test['A'] = {}
similarity_test['B'] = {}
similarity_test['A'][1] = 1
similarity_test['A'][2] = 2
similarity_test['A'][3] = 1
similarity_test['A'][4] = 7
similarity_test['A'][5] = 8
similarity_test['B'][1] = 7
similarity_test['B'][2] = 8
similarity_test['B'][3] = 9
similarity_test['B'][7] = 2
similarity_test['B'][8] = 3

print similarity_test
pearson_similarity(similarity_test['A'], similarity_test['B'])

{'A': {1: 1, 2: 2, 3: 1, 4: 7, 5: 8}, 'B': {8: 3, 1: 7, 2: 8, 3: 9, 7: 2}}


-0.9284766908852594

In [71]:
reviews['score'][0] / 10.0

6.9000000000000004

In [80]:
num_reviews = len(reviews)

item_reviews = {}

for i in range(0, num_reviews):

    item_id = reviews['uniqueID'][i]
    
    critic_review = reviews['reviewType'][i].encode('utf-8')

    if critic_review == 'c':
        user = reviews['publication'][i].encode('utf-8')
        score = reviews['score'][i] / 10.0
    else:
        user = reviews['author'][i].encode('utf-8')
        score = reviews['score'][i]

    if pd.isnull(item_id) == False:
        if item_id not in item_reviews:
            item_reviews[item_id] = {}
        item_reviews[item_id][user] = score

In [83]:
print len(item_reviews)


for key in item_reviews.keys():  ## creates a list of all keys
    if len(item_reviews[key]) < 100:
        del item_reviews[key]
           
print len(item_reviews)

27530
1326


In [84]:
(1326 * 1326) * .005 // 3600

2.0

In [85]:
itemlist = []
for key in item_reviews:
    itemlist.append(key)

In [86]:
len(itemlist)

1326

In [87]:
item_reviews[3]

{'3DJuegos': 7.5,
 'AaronAction': 8,
 'AdamKnowlesy': 6,
 'Atomix': 6.5,
 'Attack of the Fanboy': 5.0,
 'BFLegend': 9,
 'BilboGames': 8,
 'Bu1k4': 9,
 'CalmDownTom': 5.0,
 'Captiosus': 4,
 'Critical Hit': 4.5999999999999996,
 'DJRickyDAus': 10,
 'DJ_Edyo': 10,
 'Dafenix': 1,
 'DannyMiami': 10,
 'DarkZero': 4.0,
 'DarwinTheCat': 3,
 'DatSimple1211': 5,
 'Dejan': 9,
 'DerWieland': 5,
 'Destructoid': 7.5,
 'Digital Spy': 6.0,
 'Digitally Downloaded': 6.0,
 'Dog4life': 4,
 'EGM': 6.0,
 'Edge Magazine': 5.0,
 'ElektroDragon': 6,
 'ElementalSheep': 10,
 'ElfishPresley': 5,
 'Enforcer122': 2,
 'Ericistooop': 6,
 'Etreiw': 8,
 'Eurogamer Italy': 6.0,
 'Eurogamer Portugal': 5.0,
 'Fodan10': 0,
 'FreddyFiasco': 8,
 'FrontierGames': 1,
 'G0ddeity': 9,
 'GAMERGUY25': 7,
 'GOODFELLA87': 0,
 'Game Informer': 7.0,
 'Game Over Online': 7.2000000000000002,
 'Game Revolution': 6.0,
 'Game World Navigator Magazine': 6.7999999999999998,
 'GameOver.gr': 6.0,
 'GameSpot': 5.0,
 'GameTrailers': 6.20000000000

In [88]:
review_sims = pd.DataFrame(index = itemlist, columns = itemlist)

In [89]:
review_sims.head()

,3,25,51,73,95,171,182,253,313,337,...,27128,27142,27155,27293,27356,27367,27388,27404,27595,27723
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:

for i in range(0, len(review_sims.index)):
    for j in range(0, len(review_sims.index)):
        item1 = review_sims.index[i]
        item2 = review_sims.columns[j]

        if i == j:
            review_sims.iloc[i, j] = 1
        
        else:
            review_sims.iloc[i, j] = pearson_similarity(item_reviews[item1], item_reviews[item2])





In [91]:
review_sims.head(10)

,3,25,51,73,95,171,182,253,313,337,...,27128,27142,27155,27293,27356,27367,27388,27404,27595,27723
3,1,0.443913,0.308666,0.0381164,0.365199,0.184312,0.078321,0.717126,0.847922,0,...,0,0,0,0,0,0,0,0,0,0.791387
25,0.443913,1,-0.00955871,-0.235418,-0.036264,-0.206055,0.187101,0.862439,0,0.00663741,...,0,0,0,0,0,0,0,0,0,0
51,0.308666,-0.00955871,1,0.340238,0.262468,0.257613,-0.0237654,0.609682,0.255282,0,...,0,0,0,0,0,0,0,0,0,0
73,0.0381164,-0.235418,0.340238,1,0.326446,0.369903,0.366808,0.204226,0.467417,0,...,0,0,0,0,0,0,0,0,0,0
95,0.365199,-0.036264,0.262468,0.326446,1,0.305,0.152899,0.593271,0.929063,0,...,0,0,0,0,0,0,0,0,0,0
171,0.184312,-0.206055,0.257613,0.369903,0.305,1,0.282427,-0.30922,0.627763,0.526075,...,0,0,0,0,0.754287,0,0,0,0,0
182,0.078321,0.187101,-0.0237654,0.366808,0.152899,0.282427,1,0.35344,0.403403,0.221095,...,0,0,0,0,0,0,0,0,0,0
253,0.717126,0.862439,0.609682,0.204226,0.593271,-0.30922,0.35344,1,0.158351,0.155369,...,0,0,0,0.0268239,0,0,0,0,0,0
313,0.847922,0,0.255282,0.467417,0.929063,0.627763,0.403403,0.158351,1,0.769768,...,0,0,0,0,0,0,0,0,0,0
337,0,0.00663741,0,0,0,0.526075,0.221095,0.155369,0.769768,1,...,0,0,0.531222,0,0,0.324803,0,0,0,0


In [120]:
#save item-item matrix
import pickle
#pickle.dump(review_sims, open( "item_item_matrix.pickle", "wb" ))

with open('item_matrix.pickle', 'wb') as handle:
    pickle.dump(review_sims, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [119]:
#enter ID and number of recommendation to recommend i.e. 24, 10 for id 24 and return top 10

def recommend(uniqueID, return_num):
    try:
        index_list = []
        similarity_list = []
        sorted_table = review_sims.sort_values([uniqueID], ascending=[False])
        index_list = sorted_table.index.values[1:return_num + 1]
        similarity_list = sorted_table[uniqueID][1:return_num + 1]
        
        index = pd.DataFrame(index_list)
        similarity = pd.DataFrame(sorted_table[uniqueID][1:return_num + 1], columns=['pearson'])
        df = pd.concat([index, similarity], axis=1
        return similarity
    except:
        return 0
    

SyntaxError: invalid syntax (<ipython-input-119-59e9f9b24a11>, line 14)

In [118]:
recommend(654, 10)


,0,pearson
0,15250,NaN
1,15023,NaN
2,17142,NaN
3,18226,NaN
4,17571,NaN
5,7040,NaN
6,16684,NaN
7,16665,NaN
8,17046,NaN
9,18400,NaN


In [ ]:
pd.DataFrame(sorted_table[item][1:amount + 1], columns=['pearson'])
    df = pd.concat([index_list, similarity_list], axis=1)